# Data Loading, Storage, 

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Reading and Writing Data in Text Format

In [2]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# 等价 于 使用 read_table 并指定分隔符为 逗号
pd.read_table('examples/ex1.csv', sep=',')
pd.read_table('examples/ex1.csv', delimiter=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# 该文件 没有 标题行，读取 这种 文件有两种方式：让 pandas 为其分配默认的列名；自己定义列名
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [6]:
# header 用作列名的行号，默认为 0 （第一行），如果文件 没有 header行 就 应该设置为 None
pd.read_csv('examples/ex2.csv', header=None) # 分配 默认的列名
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message']) # 自定义 列名

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
# 将 message 列 作为DataFrame的索引
names = ['a', 'b', 'c', 'd', 'message']
# 通过 names 来自定义列名； 通过 index_col 明确表示将该列放到索引的位置上
# index_col 用作 行索引 的列编号 或者 列名，可以使单个 名称/数字 或由 多个名称/数字 组成的列表（层次化索引）
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [8]:
!cat examples/csv_mindex.csv
# 如果 希望将多个列 做成 一个层次化索引，只需要传入由 列编号或列名组成的 列表即可
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [9]:
# 有些表格 可能不是用 固定的分隔符 去分隔字段的（比如用空格符 或其他模式）
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [10]:
# 使用正则表达式 来作为read_table 的 分隔符；
# 此处因为 列名比数据行的数量少，read_table 推断 第一列应该是DataFram的索引
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [11]:
# 异型文件格式
# skiprows 跳过文件的 第一行、第三行、第四行
!cat examples/ex4.csv
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [12]:
# 缺失值 处理
!cat examples/ex5.csv
result = pd.read_csv('examples/ex5.csv')
result
pd.isnull(result)

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [13]:
# na_values 可以接受一组用于表示 缺失值 的字符串
# na_values 一组用于替换 NA的值
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [14]:
!cat examples/ex5.csv
# 使用字典 为 各列 指定不同的 NA 标记值
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces（逐块读取文本文件）

In [15]:
# 设置 要展示的行 数
pd.options.display.max_rows = 10

In [16]:
#!cat examples/ex6.csv
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [17]:
# nrows 需要读取的行数，如果 只想 读取几行（避免读取这个文件）
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [18]:
# 要逐块读取文件，要设置 chunksize（行数），chunksize 文件块的大小（用于迭代）
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker # pandas.io.parsers.TextFileReader 

In [19]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

# 迭代处理 ex6.csv, 将值 计数聚合到 ‘key’列中
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [20]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing Data to Text Format（将数据写出到文本文件）

In [21]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [22]:
# DataFrame 的 to_csv方法 可以将数据写到一个 以逗号分隔的文件中
data.to_csv('examples/out.csv')
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [23]:
# 这里 直接写出 到 sys.stdout，所以仅仅是 打印出文本结果而已
# 使用 其他分隔符
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [24]:
# 缺失值 在结果中 被 表示 为 空字符串，若将其表示为其他的值，使用 na_rep 选项
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [25]:
# 如果没有设置其他选项，则会写出  行和列的标签
# 禁用 行和列 的标签
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [26]:
# 写出部分列，按照指定顺序排列
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [27]:
# Series 的 to_csv方法
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/tseries.csv')
!cat examples/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### Working with Delimited Formats（手工处理 分隔符格式）

In [28]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [29]:
import csv
f = open('examples/ex7.csv')

reader = csv.reader(f)

In [30]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [31]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [32]:
header, values = lines[0], lines[1:]

In [33]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

reader = csv.reader(f, dialect=my_dialect)

reader = csv.reader(f, delimiter='|')

with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

### JSON Data

In [34]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [35]:
# 将json字符串转化为 python形式： json.loads(obj) # obj: json string
# 将 python 对象 转化为 json格式： json.dumps(result) # python 对象 
import json
result = json.loads(obj)
result # python 对象（字典）

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [36]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [37]:
# 将 json 转化为 便于分析的数据结构 或 DataFrame 
# 向 DataFrame 构造器传入 一组 json对象，并选取 数据字段的子集（或者全集）
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [38]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [39]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [40]:
print(data.to_json())
print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML and HTML: Web Scraping （web信息收集）

conda install lxml
pip install beautifulsoup4 html5lib

In [41]:
from lxml.html import parse

In [42]:
# python 可以多 HTML 和 XML格式的库 
# lxml：能够高效且可靠地解析大文件
# 首先，我要用 lxml.html处理HTML；然后，再用 lxml.objectify 做些 XML处理
tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)
failures = tables[0]
failures.head()

1

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [43]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

#### Parsing XML with lxml.objectify（利用 lxml.objectify 解析 xml）

<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>

In [44]:
from lxml import objectify
# XML（Extensible Markup Language）另一种常见的 支持分层、嵌套数据以及元数据的结构化数据格式
path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
# 通过 getroot 得到 该XML文件的根节点的引用
root = parsed.getroot() # lxml.objectify.ObjectifiedElement

In [45]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']
# root.INDICATOR 用于产生 各个<INDICATOR> XML 元素的生成器
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [46]:
# 将这组字典 转化为 一个 DataFrame
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [47]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [48]:
root
root.get('href')
root.text

<Element a at 0x7f5b84a17908>

'http://www.google.com'

'Google'

## Binary Data Formats（二进制数据格式）

In [49]:
# 实现数据的二进制格式存储 最简单的方式 之一 就是使用 python 内置的  pickle 序列化
# pandas 对象都有一个 用于将数据 pickle 形式保存到磁盘的 save方法：to_pickle
frame = pd.read_csv('examples/ex1.csv')
frame
frame.to_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [50]:
# pickle 仅 建议 用于短期 存储格式
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [51]:
!rm examples/frame_pickle

### Using HDF5 Format


##### HDF5(hierarchical data format)层次型数据格式
一个流行的工业级库，可以实现高效的读写 磁盘上 以 二进制格式存储的 科学数据
每个HDF5文件 都包含一个 文件系统式 的节点结构，它使你能够存储多个数据集并支持 元数据；

相比其他格式：

    1. HDF5支持多种 压缩器的即时压缩，还能更高效地存储重复 模式数据。
    2. 对于那些非常大的无法直接放入内存的数据集，HDF5可以高效地分块读写。
python 中的HDF5库 提供两个接口（PyTables 和 h5py），他们各自采取了不同的问题解决方式：

    1. h5py提供了一种直接 而高级的 HDF5 Api 访问接口；
    2. PyTables 则抽象了HDF5的许多细节以提供 多种灵活的数据容器、表索引、查询功能以及 对 核外计算技术（out-of-core computation）的默写支持
    
pandas 有一个 最小化 的 类似于 字典的 HDFStore 类，它通过 PyTables存储 pandas对象

HDF5 能过显著提高IO密集型（非 CPU密集型）应用的效率。


注意：HDF5并非是数据库，最适合用作“一次写多次读的数据集”，虽然数据可以在任何时候被添加到文件中，
但是如果同时发生多个写操作，文件就可能会被破坏

In [59]:
# HDF5(hierarchical data format)层次型数据格式：一个流行的工业级库，可以实现高效的读写 磁盘上 以 二进制格式存储的 科学数据
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [60]:
# HDF5 中的 对象 可以通过 与 字典一样的方式 进行 获取
store['obj1']

,a
0,0.758202
1,-0.515583
2,-0.591202
3,0.896746
4,-0.971438
...,...
95,-0.885425
96,-1.280893
97,0.196110
98,0.954644


In [64]:
store.put('obj2', frame, format='table')
# 选取 [10,15]行数据 
store.select('obj2', where=['index >= 10 and index <= 15'])
store.close()

ClosedFileError: mydata.h5 file is not open!

In [75]:
import os,sys
# 保存 到本地
frame.to_hdf('mydata.h5', 'obj3', format='table')

In [76]:
# 从本地读取 符合条件的数据
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

ValueError: The file 'mydata.h5' is already opened, but not in read-only mode (as requested).

In [77]:
os.remove('mydata.h5')

### Reading Microsoft Excel Files

In [79]:
# 读取 ExcelFile 用到了 xlrd 和 openpyxl包
# 创建 一个 ExcelFile 实例
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [81]:
# 使用 pandas中的 read_excel 方法，传入 ExcelFile对象， 表格
pd.read_excel(xlsx, 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [84]:
# 两种方式 等价 
frame1 = xlsx.parse('Sheet1')
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame1
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [86]:
# 进行写操作，使用 ExcelWriter对象
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [89]:
# 直接 使用 pandas 的 to_excel方法，
frame.to_excel('examples/ex2.xlsx')

In [90]:
!rm examples/ex2.xlsx

## Interacting with Web APIs（使用 HTML和 Web API）

In [92]:
import requests
# 发送一个 HTTP Get请求
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp  # <Response [200]> Response 对象的text属性含有 GET请求的 内容；

<Response [200]>

In [98]:
import json
data1 = json.loads(resp.text)  # load 则涉及到文件
type(data1)

list

In [99]:
# 许多 Web API返回的都是 JSON字符串，我们必须将其加载到一个 Python对象中
data = resp.json()
data[0]['title']

'pd.to_datetime() flaw mixed behavior yielding inconsistant results'

In [100]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,27454,pd.to_datetime() flaw mixed behavior yielding ...,[],open
1,27453,Right merge not preserve order,[],open
2,27451,`Dataframe.sample()` isn't validating `frac`,[],open
3,27450,Why read_sas is converting strings to float?,[],open
4,27447,WIP: treat list of namedtuples like list of di...,[],open
...,...,...,...,...
25,27414,read_csv memory leak,"[{'id': 307649777, 'node_id': 'MDU6TGFiZWwzMDc...",open
26,27413,CLN/REF: Unify Arithmetic Methods,"[{'id': 76865106, 'node_id': 'MDU6TGFiZWw3Njg2...",open
27,27411,REF: stop allowing iNaT in TimedeltaBlock methods,"[{'id': 76865106, 'node_id': 'MDU6TGFiZWw3Njg2...",open
28,27408,CLN: get parts of Block.replace out of try/except,[],open


## Interacting with Databases（使用数据库）

In [101]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [102]:
# 插入 几条数据
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [104]:
# 在 表中 选取数据时，大部分Python SQL驱动器（PyODBC，psycopg2、MySQLdb、pymssql等）都会返回一个元组列表
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [110]:
# 可以 将这个 元组列表 传给 DataFrame 的构造器，但 还需要列名（位于 游标的description属性中）
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])
# pd.DataFrame(rows, columns=zip(*cursor.description)[0]) # 需要查找一下 迭代器 如何取值？？？

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [114]:
# 如上所示，这种规整 很多，肯定不想没查询一次数据库，就重写一次啊
# pandas 有一个简化该过程的 read_frame函数（位于 pandas.io.sql模块）
# 只需要 传入 select 语句 和 连接 对象即可
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

# 第二种方式：
import pandas.io.sql as sql
sql.read_sql_query('select * from test', con)
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [116]:
!rm mydata.sqlite

rm: cannot remove 'mydata.sqlite': No such file or directory


## Conclusion